In [1]:
import pandas as pd
import numpy as np

import keras
import keras.utils
import spacy

from keras.callbacks import TensorBoard
from keras import optimizers

from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional
from keras.utils import pad_sequences
from keras_preprocessing import sequence

In [2]:
train_data = pd.read_csv('processed_train_data.tsv', sep='\t', index_col=0)
val_data = pd.read_csv('processed_val_data.tsv', sep='\t', index_col=0)
test_data = pd.read_csv('processed_test_data.tsv', sep='\t', index_col=0)

In [3]:
def format_string2np(string_numpy):
    """formatting : Conversion of String List to List

    Args:
        string_numpy (str)
    Returns:
        l (list): list of values
    """
    list_values = string_numpy.strip('[]').split(', ')
    return np.array(list_values).astype(int)

In [4]:
columns = ['word_id_custom', 'word_id_spacy', 'pos_id', 'pos_id_DEFAULT']

for column in columns:
    train_data[column] = train_data[column].apply(format_string2np)
    val_data[column] = val_data[column].apply(format_string2np)
    test_data[column] = test_data[column].apply(format_string2np)

In [5]:
embedding_matrix_custom = np.load('embedding_matrix_custom.npy')
embedding_matrix_spacy = np.load('embedding_matrix_spacy.npy')
embedding_shape = embedding_matrix_custom.shape

print("Shape of the embedding matrix:", embedding_shape)

Shape of the embedding matrix: (9607, 100)


In [6]:
import pickle
vocabulary_dict_custom = pickle.load(open('vocabulary_statement_custom.p', 'rb'))
vocabulary_dict_spacy = pickle.load(open('vocabulary_statement_spacy.p', 'rb'))
vocab_length = len(vocabulary_dict_custom)

print("Length of the vocabulary dictionary:", vocab_length)

Length of the vocabulary dictionary: 9606


In [10]:
train_data.head()

,label,statement,subject,speaker,job title,state info,party,barely true,false,half-true,mostly-true,pants-on-fire,output,pos_id,pos_id_DEFAULT,statement_custom,statement_spacy,word_id_custom,word_id_spacy
index,,,,,,,,,,,,,,,,,,,
0,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,-2,"[1, 14, 8, 8, 2, 0, 1, 2, 10, 0, 0, 4, 0, 10]","[16, 5, 11, 11, 0, 7, 16, 0, 12, 7, 7, 1, 7, 12]",say annies list political group support third ...,say annies list political group support trimes...,"[3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...","[1, 5315, 633, 423, 332, 37, 3919, 120, 936]"
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,1,"[7, 1, 14, 0, 4, 0, 0, 10, 5, 1, 7, 2, 0, 1, 4...","[14, 16, 5, 7, 1, 7, 7, 12, 10, 16, 14, 0, 7, ...",when do decline coal start start when natural ...,decline coal start start natural gas take star...,"[37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...","[720, 773, 249, 249, 891, 204, 46, 249, 527, 1..."
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,2,"[8, 8, 1, 4, 8, 8, 10, 4, 1, 12, 1, 8, 8, 14, ...","[11, 11, 16, 1, 11, 11, 12, 1, 16, 9, 16, 11, ...",hillary clinton agree john mccain vote give ge...,hillary clinton agree john mccain vote george ...,"[104, 69, 734, 160, 201, 18, 89, 262, 137, 258...","[74, 49, 649, 125, 157, 12, 212, 103, 208, 274..."
3,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,-2,"[0, 0, 0, 0, 11, 2, 12, 1, 2, 0, 0, 0, 10]","[7, 7, 7, 7, 3, 0, 9, 16, 0, 7, 7, 7, 12]",health care reform legislation be likely manda...,health care reform legislation likely mandate ...,"[19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...","[13, 16, 165, 202, 514, 342, 301, 372, 140, 2747]"
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,1,"[14, 2, 0, 1, 4, 14, 0, 4, 5, 0, 10]","[5, 0, 7, 16, 1, 5, 7, 1, 10, 7, 12]",economic turnaround start end term,economic turnaround start end term,"[282, 3331, 308, 247, 248]","[224, 3208, 249, 198, 199]"


## to test the different preprocessing methods, the following experiments will be run:
1. siddarth - baseline

        - ogdict
        - nltk stopword removal only

****
****

## todo
****
### INITIALISE INPUT/OUTPUT
1. [x] word_id custom AS X_train_custom
  - [x] embedding_matrix_custom
  - [x] vocabulary_dict_custom
2. [x] word_id spacy AS X_train_spacy
  - [x] embedding_matrix_spacy
  - [x] vocabulary_dict_spacy
  
3. [x] pos_id custom AS X_train_pos_custom
4. [x] pos_id spacy AS X_train_pos_spacy
    
#### Input variables to be processed: 
- [ ] meta
- [ ] dep parse

****
### GENERAL
- [x] change everything to python 3

### variables, init, etc.
- [ ] pass in vocabulary.p
****
### BILSTM MODEL
- [ ] functions: train(), etc
        (CODE CELLS COULD BE BETTER, INVESTIGATE)
- [ ] black box everything to understand la
- [ ]

****
*decide which word id to use*

1. fathan - pos tag check [ ]
    - custom pos tag dict
    - spacy preprocess - spacy word id
    - glove
 2. fathan - preprocess [ ]
    - custom pos tag dict
    - custom preprocess - custom word id
    - glove
    * check best results with default pos tag [ ]
****

# HYPERPARAMS

In [7]:
EMBED_DIM = 100

# vocab_length = len(vocabulary_dict.keys())
custom_vocabLen = len(vocabulary_dict_custom.keys())
spacy_vocabLen = len(vocabulary_dict_spacy.keys())
hidden_size = EMBED_DIM #Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

## TODO: preprocess metadata
#Meta data related hyper params
# num_party = len(train_data.party_id.unique())
# num_state = len(train_data.state_id.unique())
# num_venue = len(train_data.venue_id.unique())
# num_job = len(train_data.job_id.unique())
# num_sub = len(train_data.subject_id.unique())
# num_speaker = len(train_data.speaker_id.unique())

# print num_party
# print num_state
# print num_venue
# print num_job
# print num_sub
# print num_speaker
# print train_data.columns

In [8]:
print(train_data['word_id_custom'])

index
0        [3, 5440, 717, 493, 396, 54, 274, 4039, 155, 1...
1        [37, 9, 804, 861, 308, 308, 37, 981, 254, 39, ...
2        [104, 69, 734, 160, 201, 18, 89, 262, 137, 258...
3        [19, 22, 209, 252, 1, 592, 406, 361, 439, 176,...
4                               [282, 3331, 308, 247, 248]
                               ...                        
10235    [1, 161, 133, 5206, 314, 84, 86, 1, 408, 212, ...
10236       [157, 2, 52, 283, 213, 464, 1399, 653, 4, 389]
10237    [3, 2024, 154, 110, 1205, 3983, 124, 38, 2, 42...
10238       [1604, 28, 13, 2857, 3115, 4, 153, 1391, 1473]
10239    [216, 1429, 2087, 2, 5142, 826, 494, 327, 441,...
Name: word_id_custom, Length: 10240, dtype: object


In [62]:
# print(X_train_custom)

[[   3 5440  717 ...    0    0    0]
 [  37    9  804 ...   16  262  517]
 [ 104   69  734 ...    0    0    0]
 ...
 [   3 2024  154 ...   24 1311 1173]
 [1604   28   13 ...    0    0    0]
 [ 216 1429 2087 ... 1195  430  184]]


In [11]:
def input_pad_sequences(data):
    padded_data = sequence.pad_sequences(data, maxlen=num_steps, padding='post', truncating='post')
    return padded_data

X_train_custom = input_pad_sequences(train_data['word_id_custom'])
X_val_custom = input_pad_sequences(val_data['word_id_custom'])
X_test_custom = input_pad_sequences(test_data['word_id_custom'])

X_train_spacy = input_pad_sequences(train_data['word_id_spacy'])
X_val_spacy = input_pad_sequences(val_data['word_id_spacy'])
X_test_spacy = input_pad_sequences(test_data['word_id_spacy'])

X_train_pos = input_pad_sequences(train_data['pos_id'])
X_val_pos = input_pad_sequences(val_data['pos_id'])
X_test_pos = input_pad_sequences(test_data['pos_id'])

X_train_pos_DEFAULT = input_pad_sequences(train_data['pos_id_DEFAULT'])
X_val_pos_DEFAULT = input_pad_sequences(val_data['pos_id_DEFAULT'])
X_test_pos_DEFAULT = input_pad_sequences(test_data['pos_id_DEFAULT'])

In [12]:
# Initialise input/output

## OUTPUT
Y_train = train_data['output']
Y_train = keras.utils.to_categorical(Y_train, num_classes=6)

Y_val = val_data['output']
Y_val = keras.utils.to_categorical(Y_val, num_classes=6)

In [18]:
# # CUSTOM word_id PICKLE
# ##
# X_train_custom = pickle.load(open('train_word_id_custom.p', 'rb'))
# X_val_custom = pickle.load(open('val_word_id_custom.p', 'rb'))
# X_test_custom = pickle.load(open('test_word_id_custom.p', 'rb'))
#
# X_train_custom = sequence.pad_sequences(X_train_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
# X_train_custom = sequence.pad_sequences(X_train_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
# X_train_custom = sequence.pad_sequences(X_train_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
#
# X_train_spacy = pickle.load(open('train_word_id_spacy.p', 'rb'))
# # X_val_custom = sequence.pad_sequences(X_val_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
# # X_test_custom = sequence.pad_sequences(X_val_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
#
#
# # Y_train_custom = train_data['output']
# # Y_train_custom = keras.utils.to_categorical(Y_train_custom, num_classes=6)
#
# # Y_val_custom = val_data['output']
# # Y_val_custom = keras.utils.to_categorical(Y_val_custom, num_classes=6)
#
# # X_train_custom = sequence.pad_sequences(X_train_custom, maxlen=num_steps, padding='post',truncating='post')
# # X_val_custom = sequence.pad_sequences(X_val_custom, maxlen=num_steps, padding='post',truncating='post')

In [20]:
print(X_test_custom)
print(X_test_spacy)
print(X_test_pos)
print(X_test_pos_DEFAULT)

[[ 350  425   28 ...    0    0    0]
 [  64    1 1930 ...    0    0    0]
 [   3  160  201 ...    0    0    0]
 ...
 [ 472 1547  191 ...  142  524  372]
 [   3 1406  928 ...   78    0    0]
 [   3   93    1 ...  258   43  107]]
[[ 291  358   19 ...    0    0    0]
 [  45 1829  234 ...    0    0    0]
 [   1  125  157 ...    0    0    0]
 ...
 [ 405 1448  153 ...  106  451  310]
 [   1 1306  836 ...    0    0    0]
 [   1   64   34 ... 1571   89   77]]
[[ 1 14  0 ...  0  0  0]
 [ 8 11  4 ... 10  0  0]
 [ 1  8  8 ...  0  0  0]
 ...
 [ 4 14  2 ...  2  0  1]
 [ 1 14  8 ...  1  5  1]
 [ 1 14  0 ... 14  5 11]]
[[16  5  7 ...  0  0  0]
 [11  3  1 ... 12  0  0]
 [16 11 11 ...  0  0  0]
 ...
 [ 1  5  0 ...  0  7 16]
 [16  5 11 ... 16 10 16]
 [16  5  7 ... 13 10  3]]


In [13]:
# CUSTOM word_id numpy
##

# X_train_custom = pd.DataFrame(train_data['word_id_custom'])
# X_val_custom = val_data['word_id_custom']
# X_test_custom = test_data['word_id_custom']


# X_train_custom = pickle.load(open('train_word_id_custom.p', 'rb'))
X_train_custom_np = np.load('train_word_id_custom.npy', allow_pickle=True)
# X_val_custom = val_data['word_id_custom'].apply(lambda x: x.split()).tolist()
# X_test_custom = test_data['word_id_custom'].apply(lambda x: x.split()).tolist()


# X_train_custom = sequence.pad_sequences(X_train_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
X_train_custom_np = sequence.pad_sequences(X_train_custom_np, maxlen=num_steps, dtype=object, padding='post', truncating='post')
# X_val_custom = sequence.pad_sequences(X_val_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')
# X_test_custom = sequence.pad_sequences(X_val_custom, maxlen=num_steps, dtype=object, padding='post', truncating='post')


Y_train_custom = train_data['output']
Y_train_custom = keras.utils.to_categorical(Y_train_custom, num_classes=6)

Y_val_custom = val_data['output']
Y_val_custom = keras.utils.to_categorical(Y_val_custom, num_classes=6)

# X_train_custom = sequence.pad_sequences(X_train_custom, maxlen=num_steps, padding='post',truncating='post')
# X_val_custom = sequence.pad_sequences(X_val_custom, maxlen=num_steps, padding='post',truncating='post')

In [15]:
# print(X_train_custom_pd)
print(X_train_custom_np)
print(X_train_custom)

[[3 5440 717 ... 0.0 0.0 0.0]
 [37 9 804 ... 16 262 517]
 [104 69 734 ... 0.0 0.0 0.0]
 ...
 [3 2024 154 ... 24 1311 1173]
 [1604 28 13 ... 0.0 0.0 0.0]
 [216 1429 2087 ... 1195 430 184]]
[[3 5440 717 ... 0.0 0.0 0.0]
 [37 9 804 ... 16 262 517]
 [104 69 734 ... 0.0 0.0 0.0]
 ...
 [3 2024 154 ... 24 1311 1173]
 [1604 28 13 ... 0.0 0.0 0.0]
 [216 1429 2087 ... 1195 430 184]]


In [14]:
# SPACY word_id
##
X_train_spacy = train_data['word_id_spacy'].apply(lambda x: x.split()).tolist()
X_val_spacy = val_data['word_id_spacy'].apply(lambda x: x.split()).tolist()
X_test_spacy = test_data['word_id_spacy'].apply(lambda x: x.split()).tolist()

X_train_spacy = keras.utils.pad_sequences(X_train_spacy, maxlen=num_steps, dtype=object, padding='post',truncating='post')
X_val_spacy = keras.utils.pad_sequences(X_val_spacy, maxlen=num_steps, dtype=object, padding='post',truncating='post')

Y_train_spacy = train_data['output']
Y_train_spacy = keras.utils.to_categorical(Y_train_spacy, num_classes=6)

Y_val_spacy = val_data['output']
Y_val_spacy = keras.utils.to_categorical(Y_val_spacy, num_classes=6)



In [1]:
# CUSTOM POS
X_train_pos_custom = train_data['pos_id']
X_val_pos_custom = val_data['pos_id']
X_test_pos_custom = test_data['pos_id']

X_train_pos_custom = keras.utils.pad_sequences(X_train_pos_custom, maxlen=num_steps, padding='post')
X_val_pos_custom = keras.utils.pad_sequences(X_val_pos_custom, maxlen=num_steps, padding='post')
X_test_pos_custom = keras.utils.pad_sequences(X_test_pos_custom, maxlen=num_steps, padding='post')


NameError: name 'train_data' is not defined

In [ ]:
# SPACY POS
X_train_pos_spacy = train_data['pos_id_spacy']
X_val_pos_spacy = val_data['pos_id_spacy']
X_test_pos_spacy = test_data['pos_id_spacy']

X_train_pos_spacy = keras.utils.pad_sequences(X_train_pos_spacy, maxlen=num_steps, padding='post')
X_val_pos_spacy = keras.utils.pad_sequences(X_val_pos_spacy, maxlen=num_steps, padding='post')
X_test_pos_spacy = keras.utils.pad_sequences(X_test_pos_spacy, maxlen=num_steps, padding='post')


#### 2. fathan - pos tag check [ ]
    - custom pos tag dict
    - spacy preprocess - spacy word id
    - glove

In [ ]:
print(X_train_custom.shape, X_val_custom.shape, X_test_custom.shape)

In [ ]:
def train(model, name, use_pos=False, use_meta=False, use_dep=False):
    sgd = optimizers.SGD(lr=0.025, clipvalue=0.3, nesterov=True)
    adam = optimizers.Adam(lr=0.000075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    tb = TensorBoard()
    csv_logger = keras.callbacks.CSVLogger('training.log')
    filepath = name + "_weights_best.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

    if use_pos and use_meta:
        if use_dep:
            model.fit(
                {'main_input': X_train, 'pos_input': X_train_pos, 'aux_input': X_train_meta, 'dep_input': X_train_dep},
                {'main_output': Y_train},
                epochs=num_epochs,
                batch_size=batch_size,
                validation_data=(
                    {'main_input': X_val, 'pos_input': X_val_pos, 'aux_input': X_val_meta, 'dep_input': X_val_dep},
                    {'main_output': Y_val}),
                callbacks=[tb, csv_logger, checkpoint])
        else:
            model.fit(
                {'main_input': X_train, 'pos_input': X_train_pos, 'aux_input': X_train_meta},
                {'main_output': Y_train},
                epochs=num_epochs,
                batch_size=batch_size,
                validation_data=(
                    {'main_input': X_val, 'pos_input': X_val_pos, 'aux_input': X_val_meta},
                    {'main_output': Y_val}),
                callbacks=[tb, csv_logger, checkpoint])
    elif use_meta:
        if use_dep:
            model.fit(
                {'main_input': X_train, 'aux_input': X_train_meta, 'dep_input': X_train_dep},
                {'main_output': Y_train},
                epochs=num_epochs,
                batch_size=batch_size,
                validation_data=(
                    {'main_input': X_val, 'aux_input': X_val_meta, 'dep_input': X_val_dep},
                    {'main_output': Y_val}),
                callbacks=[tb, csv_logger, checkpoint])
        else:
            model.fit(
                {'main_input': X_train, 'aux_input': X_train_meta},
                {'main_output': Y_train},
                epochs=num_epochs,
                batch_size=batch_size,
                validation_data=(
                    {'main_input': X_val, 'aux_input': X_val_meta},
                    {'main_output': Y_val}),
                callbacks=[tb, csv_logger, checkpoint])
    elif use_pos:
        if use_dep:
            model.fit(
                {'main_input': X_train, 'pos_input': X_train_pos, 'dep_input': X_train_dep},
                {'main_output': Y_train},
                epochs=num_epochs,
                batch_size=batch_size,
                validation_data=(
                    {'main_input': X_val, 'pos_input': X_val_pos, 'dep_input': X_val_dep},
                    {'main_output': Y_val}),
                callbacks=[tb, csv_logger, checkpoint])
        else:
            model.fit(
                {'main_input': X_train, 'pos_input': X_train_pos},
                {'main_output': Y_train},
                epochs=num_epochs,
                batch_size=batch_size,
                validation_data=(
                    {'main_input': X_val, 'pos_input': X_val_pos},
                    {'main_output': Y_val}),
                callbacks=[tb, csv_logger, checkpoint])
    else:
        if use_dep:
            model.fit(
            {'main_input': X_train,'dep_input':X_train_dep},
            {'main_output': Y_train}, epochs = num_epochs, batch_size = batch_size,
            validation_data = (
                                {'main_input': X_val, 'dep_input':X_val_dep},
                                {'main_output': Y_val}
                                ), callbacks=[tb,csv_logger,checkpoint])
    else:
      model.fit(
        {'main_input': X_train},
        {'main_output': Y_train}, epochs = num_epochs, batch_size = batch_size,
        validation_data = (
            {'main_input': X_val},
            {'main_output': Y_val}
        ), callbacks=[tb,csv_logger,checkpoint])


In [ ]:
use_pos=False
use_meta=True
use_dep=True

# LSTM model
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_length+1, hidden_size, input_length=num_steps))
model_lstm.add(Bidirectional(LSTM(hidden_size)))
model_lstm.add(Dense(6, activation='softmax'))


# statement embed LSTM
statement_input = Input(shape=(num_steps,), dtype='int32', name='main_input')
x = Embedding(vocab_length+1,EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=False)(statement_input)
lstm_in = LSTM(lstm_size,dropout=0.2)(x)



# pos embed LSTM
pos_input = Input(shape=(num_steps,), dtype='int32', name='pos_input')
x2 = Embedding(max(pos_dict.values()), max(pos_dict.values()), weights=[pos_embeddings], input_length=num_steps, trainable=False)(pos_input)
lstm_in2 = LSTM(lstm_size, dropout=0.2)(x2)


# dep embed LSTM
dep_input = Input(shape=(num_steps,), dtype='int32', name='dep_input')
x3 = Embedding(max(dep_dict.values()), max(dep_dict.values()), weights=[dep_embeddings], input_length=num_steps, trainable=False)(dep_input)
lstm_in3 = LSTM(lstm_size, dropout=0.2)(x3)


# meta data Dense
meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = Dense(64, activation='relu')(meta_input)


if use_pos and use_meta:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in2, lstm_in3, x_meta])
  else:
    x = keras.layers.concatenate([lstm_in, lstm_in2, x_meta])
elif use_meta:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in3, x_meta])
  else:
    x = keras.layers.concatenate([lstm_in, x_meta])
elif use_pos:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in3, lstm_in2])
  else:
    x = keras.layers.concatenate([lstm_in, lstm_in2])
else:
  if use_dep:
    x = keras.layers.concatenate([lstm_in, lstm_in3])
  else:
    x = lstm_in




main_output = Dense(6, activation='softmax', name='main_output')(x)

if use_pos and use_meta:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, pos_input, dep_input, meta_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input, pos_input, meta_input], outputs=[main_output])
elif use_meta:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, dep_input, meta_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input, meta_input], outputs=[main_output])
elif use_pos:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, dep_input, pos_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input, pos_input], outputs=[main_output])
else:
  if use_dep:
    model_lstm = Model(inputs=[statement_input, dep_input], outputs=[main_output])
  else:
    model_lstm = Model(inputs=[statement_input], outputs=[main_output])

In [ ]:
print model_lstm.summary()